In [1]:
import pandas as pd
import numpy as np

In [15]:
!wget https://github.com/Moradnejad/ColBERT-Using-BERT-Sentence-Embedding-for-Humor-Detection/raw/master/Data/dataset.csv
!wget https://github.com/Moradnejad/ColBERT-Using-BERT-Sentence-Embedding-for-Humor-Detection/raw/master/Data/test.csv
!wget https://github.com/Moradnejad/ColBERT-Using-BERT-Sentence-Embedding-for-Humor-Detection/raw/master/Data/train.csv
!wget https://github.com/Moradnejad/ColBERT-Using-BERT-Sentence-Embedding-for-Humor-Detection/raw/master/Data/dev.csv

--2023-02-01 14:19:19--  https://github.com/Moradnejad/ColBERT-Using-BERT-Sentence-Embedding-for-Humor-Detection/raw/master/Data/dataset.csv
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Moradnejad/ColBERT-Using-BERT-Sentence-Embedding-for-Humor-Detection/master/Data/dataset.csv [following]
--2023-02-01 14:19:19--  https://raw.githubusercontent.com/Moradnejad/ColBERT-Using-BERT-Sentence-Embedding-for-Humor-Detection/master/Data/dataset.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14874240 (14M) [text/plain]
Saving to: ‘dataset.csv.1’

dataset.csv.1       100%[===================>]  14.1

In [3]:
data = pd.read_csv('dataset.csv')

test_texts = pd.read_csv('test.csv')['text'].tolist()


data = data[data['text'].isin(test_texts)]

In [4]:
df = pd.read_csv('test.csv')

In [5]:
df

,text
0,What kind of cat should you take into the des...
1,Remember when people used to have to be in sha...
2,Pizza is always good. - everyone we'll see abo...
3,"What's 6 inches long hard, bent, and in my pan..."
4,Black teen's response to violence in his commu...
...,...
39995,Conor maynard seamlessly fits old-school r&b h...
39996,How to you make holy water? you boil the hell ...
39997,How many optometrists does it take to screw in...
39998,Mcdonald's will officially kick off all-day br...


In [6]:
ann_columns = ['humor']

In [7]:
for col in ann_columns:
    data[col] = data[col].astype(int)

In [8]:
np.random.seed(22)

selected_texts = data['text'].drop_duplicates().sample(n=1000).tolist()
selected_data = data.loc[data['text'].isin(selected_texts)]
not_selected_data = data.loc[~data['text'].isin(selected_texts)]

In [9]:
def get_prompt(text):
    pattern = f'Which one of the attributes: "funny", "not funny" describes a given text? Write your answer in the form of a Python list containing the appropriate attribute.\n\nText: {text}'

    return pattern

selected_data.loc[:, 'prompt'] = selected_data['text'].apply(get_prompt)

/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


In [10]:
def get_labels(row):
    return np.array(['not funny', 'funny'][int(row[ann_columns].values == 1)])

selected_data.loc[:, 'annotation'] = selected_data.apply(get_labels , axis=1)

selected_data = selected_data.rename(columns={'text': 'comment'})

In [11]:
selected_data.loc[:, ['prompt', 'annotation', 'comment']].reset_index(drop=True).to_csv("colbert_prompts.csv")

In [12]:
data.humor.value_counts()

1    20082
0    19918
Name: humor, dtype: int64

In [13]:
from scipy.stats import entropy

def get_class(class_name):
  return ['not funny', 'funny'].index(class_name)

def _entropy(labels, base=None):
    _, counts = np.unique(labels, return_counts=True)
    return entropy(counts, base=base)

_entropy([get_class(elem) for elem in selected_data.annotation.tolist()])

0.6929851718108113

In [14]:
from scipy.stats import entropy

def get_class(class_name):
  return ['not funny', 'funny'].index(class_name)

def _entropy(labels, base=None):
    _, counts = np.unique(labels, return_counts=True)
    return entropy(counts, base=base)


_entropy([elem for elem in data[ann_columns[0]].tolist()])



0.6931387755363971

# Prepare dataset

In [64]:
train_df = pd.read_csv('train.csv')
dev_df = pd.read_csv('dev.csv')
test_df = selected_data.loc[:, ['annotation', 'comment']].reset_index(drop=True)

In [65]:
train_df['humor'] = train_df['humor'].astype(int)
dev_df['humor'] = dev_df['humor'].astype(int)

In [66]:
def binarize_label(label):
  label_dict = {'not funny': 0, 'funny': 1}
  return label_dict[str(label)]
  
test_df['annotation'] = test_df['annotation'].apply(binarize_label)

In [67]:
train_df = train_df.rename(columns={'humor': 'is_funny'})
dev_df = dev_df.rename(columns={'humor': 'is_funny'})
test_df = test_df.rename(columns={'comment': 'text', 'annotation': 'is_funny'})

In [68]:
train_df['text_id'] = [i for i in range(len(train_df))]
dev_df['text_id'] = [i for i in range(len(train_df), (len(train_df) + len(dev_df)))]
test_df['text_id'] = [i for i in range((len(train_df) + len(dev_df)), (len(train_df) + len(dev_df) + len(test_df)))]

In [69]:
train_df['split'] = 'train'
dev_df['split'] = 'val'
test_df['split'] = 'test'

In [70]:
full_df = pd.concat([train_df, dev_df, test_df]).reset_index(drop=True)

In [72]:
full_df['annotator_id'] = 0

In [74]:
full_df.split.value_counts()

train    160000
val       40000
test       1000
Name: split, dtype: int64

In [75]:
data_df = full_df[['text_id', 'text']]
annotations_df = full_df[['text_id', 'annotator_id', 'is_funny', 'split']]

In [77]:
data_df.to_csv('data.csv', index=False)
annotations_df.to_csv('annotations.csv', index=False)